<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP_2022/blob/main/02-Machine-Learning/notebooks/11-Regresion-Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Logística

En esta notebook usaremos la implementación de scikit-learn de la regresión logística. Primero veremos dos ejemplos didacticos para entender el funcionamiento del clasificador. Después, resolveremos un ejemplo usando el conjunto de datos de dígitos escritos a mano.

## Ejemplo 1(a)

Un ejemplo con datos unidimensionales

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

X, y = make_blobs(n_samples=10,
                  n_features=1,
                  centers=2,
                  random_state=17)

plt.figure()
plt.scatter(x=X,y=[0 for x in X],c=y)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X,y)

probs = lr.predict_proba(X)
probs[:,1]

In [ ]:
from math import e

def f(x):
    return w0+w1*x

def plog(x):
    return 1/(1+e**(-f(x)))

w0, w1 = lr.intercept_, lr.coef_[0]
xmin, xmax = np.min(X), np.max(X)
xs = np.linspace(xmin,xmax,100)

plt.figure()
plt.scatter(x=X,y=[0 for x in X],c=y,s=85)
plt.plot(xs,[plog(x) for x in xs],color='black')
plt.scatter(x=X,y=probs[:,1],c=y,marker='+',s=85)
plt.axhline(y=0.5,color='gray',linestyle='--')
plt.yticks([0,0.5,1])
plt.show()

## Ejemplo 1(b)

En este ejemplo, clasificamos un pequeño conjunto de datos en 2 dimensiones

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

X, y = make_blobs(n_samples=10,
                  n_features=2,
                  centers=2,
                  random_state=20)

plt.figure()
plt.scatter(X[:,0],X[:,1],c=y,s=60)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X,y)

Podemos acceder al intercepto y los coeficientes

In [ ]:
w0, w1, w2 = lr.intercept_, lr.coef_[0,0], lr.coef_[0,1]

Veamos las probabilidades

In [ ]:
probs = lr.predict_proba(X)
probs

Grafiquemos y visualicemos las probabilidades

In [ ]:
from math import e
from matplotlib import cm
from itertools import product

def f(x,y):
    return w0+w1*x+w2*y

def plog(x,y):
    return 1/(1+e**(-f(x,y)))

fig, ax = plt.subplots(subplot_kw={"projection": "3d"},dpi=300)

xmin, xmax = np.min(X[:,0]), np.max(X[:,0])
ymin, ymax = np.min(X[:,1]), np.max(X[:,1])

xs = np.linspace(xmin, xmax,50)
ys = np.linspace(ymin, ymax, 50)
Xs, Ys = np.meshgrid(xs, ys)
Z = np.array([plog(x,y) for x,y in zip(Xs,Ys)])

surf = ax.plot_surface(Xs, Ys, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.plot_surface(Xs, Ys, 0.5+np.zeros_like(Z),
                       linewidth=0, antialiased=False,alpha=0.4,color='gray')
ax.scatter(X[:,0],X[:,1],[0 for x in X],c=y,s=60)
ax.scatter(X[:,0],X[:,1],[plog(x[0],x[1]) for x in X],c='black',s=60,marker='x')
plt.show()

## Ejemplo 2(a)

En el siguiente ejemplo vemos cómo generar fronteras de decisión más complejas.

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/02-Machine-Learning/data/binary-classification-data.csv'
df = pd.read_csv(url,header=None)
df

Observemos los datos

In [ ]:
import matplotlib.pyplot as plt

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

plt.figure()
plt.scatter(X[:,0],X[:,1],c=y,s=60)
plt.show()

In [ ]:
#@title Funciones para graficas las fronteras de decisión
import numpy as np

def make_meshgrid(x, y, h=.02):
    '''
    función para hacer la malla de puntos para colorear las regiones de decisión,
    la malla de puntos abarca la región donde se encuentran los puntos (x,y)
    'h' es el tamaño de paso
    '''
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

Hacemos una regresión logística con los parámetros por default y vemos su accuracy en el conjunto de entrenamiento.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X,y)
lr.score(X,y)

Hacemos una regresión logística con polinomial features y vemos su accuracy en el conjunto de entrenamiento

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

p_feats = PolynomialFeatures(2,include_bias=False)
log_reg = LogisticRegression(penalty='l2', C=1, solver='newton-cholesky')

pl = Pipeline([('pf',p_feats),
               ('clf',log_reg)])
pl.fit(X,y)
pl.score(X,y)

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1])

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, lr, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Regresión Logística')

plot_contours(ax2, pl, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Regresión Logística con Polinomial Features')

plt.show()

## Ejemplo 2(b)

En este ejemplo veremos el efecto de la regularización como herramienta para prevenir el *overfitting*. Veremos un dataset con muchas features y varias de ellas correlacionadas.

In [ ]:
from sklearn.datasets import make_classification

# X, y = make_classification(n_samples=300,
#                            n_features=60,
#                            n_informative=20,
#                            n_repeated=3,
#                            n_redundant=20,
#                            n_classes=2,
#                            n_clusters_per_class=3,
#                            class_sep = 0.5,
#                            random_state=49)

X, y = make_classification(n_samples=500,
                           n_features=100,
                           n_informative=20,
                           n_repeated=3,
                           n_redundant=20,
                           n_classes=2,
                           class_sep = 0.5,
                           random_state=57)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75,random_state=1001)

Realicemos el entrenamiento. El rendimiento parece ser bueno en el conjunto de entrenamiento pero en el conjunto de prueba es malo. **Esta es una señal de overfitting**.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty=None)
lr.fit(X_train,y_train)
print(f"Training score: {lr.score(X_train,y_train)}")
print(f"Test score: {lr.score(X_test,y_test)}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

normas = np.array([np.linalg.norm(x) for x in lr.coef_[0]])

plt.figure()
sns.histplot(normas)
plt.show()

Usamos un clasificador con regularización

In [ ]:
lr2 = LogisticRegression(C=0.1,penalty='l1',solver='liblinear')
lr2.fit(X_train,y_train)
print(f"Training score: {lr2.score(X_train,y_train)}")
print(f"Test score: {lr2.score(X_test,y_test)}")

In [ ]:
import seaborn as sns

normas = np.array([np.linalg.norm(x) for x in lr2.coef_[0]])

plt.figure()
sns.histplot(normas)
plt.show()

Podemos ver que hay varios coeficientes que son exactamente cero. Observar que el número es parecido al número de features redundantes + número de features repetidas

In [ ]:
coefs = lr2.coef_
num_zeros = coefs[coefs==0].shape[0]
print(f"Número de coeficientes = 0: {num_zeros}")

Veamos el score ROC-AUC

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_probs = lr2.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred_probs[:,1])
from sklearn.metrics import roc_auc_scoreprint(f"ROC-AUC score: {score}")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs[:,1])

plt.figure()
plt.suptitle("Curva ROC") 
plt.plot(fpr,tpr,color='red')
plt.plot([0,1],[0,1],linestyle='--',color='gray')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.show()

## Ejemplo 3

Usaremos el dataset de dígitos escritos a mano. Tenemos dos versiones:

* Usando `keras`, es el dataset MNIST completo. Son 70,000 imágenes de $28\times 28$, divididas en 60,000 de entrenamiento y 10,000 de prueba.
* Usando `sklearn`, es una versión reducida. Son 1797 imágenes de $8\times 8$.

Por practicidad, usaremos la segunda opción.

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape, X_test.shape

Obtenemos el dataset

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
X.shape

Veamos cómo se ve una instancia del conjunto de datos

In [ ]:
X[28]

In [ ]:
X[28].reshape(8,8)

Hacemos la división en entrenamiento y prueba.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=128)
X_train.shape, X_test.shape

Cambiamos la forma, para que cada instancia sea un arreglo bidimensional

In [ ]:
import numpy as np

train_size = X_train.shape[0]
test_size = X_test.shape[0]
X_train = X_train.reshape(train_size,-1)
X_test = X_test.reshape(test_size,-1)
X_train.shape, X_test.shape

Mostramos algunas instancias de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,2))
for idx, (image, label) in enumerate(zip(X_train[:5], y_train[:5])):
    plt.subplot(1, 5, idx + 1)
    plt.imshow(np.reshape(image, (8,8)), cmap=plt.cm.gray)
    plt.xticks([])
    plt.yticks([])
    plt.title(label)

Usaremos la implementación de scikit-learn ([documentación](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).

In [ ]:
from sklearn.linear_model import LogisticRegression

⭕ Realiza el entrenamiento de un módelo de Regresión Lineal y realiza las predicciones sobre el conjunto de prueba. Reporta las métricas de rendimiento (accuracy, recall, precision, f1-score) y la mátriz de confusión con el conjunto de prueba.

Puedes usar los parámetros que gustes para el modelo.

In [ ]:
'''
Tu código
'''

In [ ]:
y_pred_probs = lr.predict_proba(X_test)
y_pred_probs.shape

Veamos las probabilidades predichas y la etiqueta predicha

In [ ]:
print(y_pred_probs[0])
print(y_pred[0])

Como son probabilidades, la suma de las componentes es 1

In [ ]:
np.sum(y_pred_probs[0])

### Validación cruzada

Usemos validación cruzada para evaluar el entrenamiento de nuestro modelo, [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score).

In [ ]:
from sklearn.model_selection import cross_val_score


scores = cross_val_score(lr, X_train, y_train, cv=5)

In [ ]:
print(scores)
print(np.mean(scores))

### ROC-AUC Score

Usaremos el area bajo la curva ROC como métrica de rendimiento del clasificador. Para esto necesitamos las probabilidades de las predicciones.

Hay varias maneras de calcularlo:

* Si el clasificador tiene un método `predict_proba`, usamos [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html).
* Si no, usamos [RocCurveDisplay.from_estimator](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html#sklearn.metrics.RocCurveDisplay.from_estimator)

In [ ]:
y_pred_probs = lr.predict_proba(X_test)
y_pred_probs.shape

Cuando se trata de un problema multiclase podemos escoger el enfoque `ovr` (one-vs-rest) o `ovo` (one-vs-one). En este caso, no se puede graficar directamente la curva ROC.

In [ ]:
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_test, y_pred_probs,multi_class='ovr')
print(f"ROC-AUC score: {score}")

### ⭕ Práctica "final" de clasificación 

Retomar el dataset de la sesión pasada y probar los siguientes clasificadores:

* Regresión Logística
* SVM
* Decision Tree
* Random Forest

Recuerda que hay algunos valores faltantes. 

1. ¿Qué clasificador tiene mejor rendimiento en este dataset? Para esto toma en cuenta el accuracy, F1-score y Cross-Validation (accuracy promedio) como métricas de rendimiento. 

Como siempre, puedes usar cualquier técnica de selección de features, escalamiento, regularización que desees.

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/02-Machine-Learning/data/diabetes.csv'
df = pd.read_csv(url,index_col=0)
df

Hay más clasificadores que es importante revisar. Con las herramientas que ya cuentas, ya puedes revisarlos por tu cuenta:

* [K-nearest neighbors classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
* [Quadratic Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)
* [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)
* ...

[Más información](https://scikit-learn.org/stable/supervised_learning.html), [comparación](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).